In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.multitasking_lib import *

#redis_conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)


In [3]:
redis_conn.pubsub_channels()

['kite_simulatorbacktest_web',
 'freedom_initfreedom',
 'trade_handlerbacktest_web',
 'order_handlerorder']

In [4]:
my_cache = cache_state('backtest_web')

DEBUG:	Cache initialized for: backtest_web


In [16]:
df = my_cache.getValue()

In [17]:
df

,stock,qty,SL %,TP %,amount,p_n_l,Total_p_n_l,low,sl,ltp,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,TCS,10,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-04-23 09:16,SCANNING,PAUSE,default,minute,day
1,WIPRO,10,1,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019-04-23 09:27,SCANNING,PAUSE,default,minute,day


In [44]:
my_cache.getValue()

,stock,qty,SL %,TP %,amount,p_n_l,Total_p_n_l,low,sl,ltp,tp,high,last_processed,state,mode,algo,freq,hdf_freq
0,TCS,10,0.3,0.3,21410.0,0.0,0.0,2139.9,2134.5769999999998,2140.05,2147.423,2142.2,2019-04-23 12:33,LONG,PAUSE,default3,minute,day
1,WIPRO,10,0.3,0.3,2887.5,0.0,0.0,287.7,287.88375,288.15,289.61625,288.4,2019-04-23 12:33,LONG,PAUSE,default3,minute,day


In [19]:
trades_df = my_cache.getOHLC('WIPRO')

In [41]:
df  = trades_df.iloc[-1:]['close'][0]
df

290.9

In [11]:
if not 'buy' in trades_df.columns:
    print('Yeah')

Yeah


In [57]:
trades_df['trade'] = -1 * trades_df['buy']
trades_df['trade'] = trades_df['sell']

In [17]:
trades_df['sell'].map("{:.02f}".format)

2019-05-02    293.65
Name: sell, dtype: object

In [85]:
(total_profit, max_loss, max_profit, total_win, total_loss, max_winning_streak, max_loosing_streak, trade_log_df) = trade_analysis_raw(trades_df)

In [4]:
######################
# Initiate Backtest  #
######################
cache_type ='backtest_uni'
def backtest_unitest(stocks):
    stocks = stocks
    sl = 1
    target = 1
    qty = 10
    algo = ""
    fromDate = '2019-5-2'
    toDate = '2019-5-3'
    freq = '1min'
    backtest_msg={
            'stock':stocks,
            'sl':sl,
            'target':target,
            'qty':qty,
            'algo':algo,
            'fromDate':fromDate,
            'toDate':toDate,
            'freq':freq
        }

    for stock in stocks:
        redis_conn.set('stock'+cache_type,stock)
        # Step 3 : Store the OHLC and backtest data in the redis: to be used by plotter
        #ohlc_dict = '[{"date":'+json.dumps(fromDate)+',"close":0,"high":0,"low":0,"open":0,"volume":0}]'
        #redis_conn.set(stock+cache_postfix,ohlc_dict)

    # Step 4: Done is set to 0: Backtest is in progress, will be resetted by backtest job
    redis_conn.set('done'+cache_type,0)

    redis_conn.publish('kite_simulator'+cache_type,json.dumps(backtest_msg))

In [11]:
##########################
# Unit Test              #
##########################

open('log/freedom.log','r+')
#log_file.close()

logger.setLevel(logging.DEBUG)
loggerT.setLevel(21)
redis_conn.set('done'+cache_type,1)
#TODO: Implement shared memory and split
backtest_manager = threadManager(cache_type, ["kite_simulator","trade_handler"], [kite_simulator, trade_handler])
#trade_manager = threadManager("trade_manager", ["trade_handler"], [trade_handler])
order_manager = threadManager("order", ["order_handler"], [order_handler])

print(backtest_manager.job.is_alive())
print(order_manager.job.is_alive())
print(redis_conn.pubsub_channels())
print(jobs)


DEBUG:	Cache initialized for: backtest_uni
DEBUG:	Cache initialized for: order
DEBUG:	Cache initialized for: order


True
True
['kite_simulatorbacktest_web', 'order_handlerorder', 'freedom_initfreedom', 'trade_handlerbacktest_web']
[<Process(Process-1, stopped[SIGTERM])>, <Process(Process-2, stopped[SIGTERM])>, <Process(Process-3, started)>, <Process(Process-4, started)>]


DEBUG:	Cache initialized for: backtest_uni
INFO:	Starting Handler: order_handler
INFO:	Starting Handler: kite_simulator
INFO:	Starting Handler: trade_handler
INFO:	Starting Handler: kite_simulator
INFO:	Starting Handler: order_handler
DEBUG:	order_handler: 1
INFO:	Starting Handler: trade_handler
DEBUG:	kite_simulator: 1
DEBUG:	kite_simulator: 1
DEBUG:	order_handler: 1
INFO:	trade_handler: 1
INFO:	trade_handler: 1
ERROR:	kite_simulator: Invalid msg: 1
ERROR:	kite_simulator: Invalid msg: 1
DEBUG:	kite_simulator: {"stock": ["WIPRO"], "sl": 1, "target": 1, "qty": 10, "algo": "", "fromDate": "2019-5-2", "toDate": "2019-5-3", "freq": "1min"}
DEBUG:	kite_simulator: {"stock": ["WIPRO"], "sl": 1, "target": 1, "qty": 10, "algo": "", "fromDate": "2019-5-2", "toDate": "2019-5-3", "freq": "1min"}
DEBUG:	Trade_init: WIPRO
DEBUG:	Trade_init: WIPRO
INFO:	WIPRObacktest_uni
INFO:	WIPRObacktest_uni
TRADE: S: EN:  : 2019-05-02 09:24 : WIPRO -> 297.75
TRADE: S: EN:  : 2019-05-02 09:24 : WIPRO -> 297.75
TRA

In [12]:
backtest_unitest(['WIPRO'])

In [7]:
backtest_unitest(['TCS'])

In [29]:
backtest_unitest(['WIPRO','TCS'])

In [13]:
logMsg = redis_conn.get('logMsg'+cache_type)
print(logMsg)

Backtest Started: WIPRO :



In [15]:
# Terminate all the processes
backtest_manager.job.terminate()
order_manager.job.terminate()
time.sleep(0.1)
print(backtest_manager.job.is_alive())
print(order_manager.job.is_alive())
print(redis_conn.pubsub_channels())
jobs

False
False
['kite_simulatorbacktest_web', 'order_handlerorder', 'freedom_initfreedom', 'trade_handlerbacktest_web']


[<Process(Process-1, stopped[SIGTERM])>,
 <Process(Process-2, stopped[SIGTERM])>,
 <Process(Process-3, stopped[SIGTERM])>,
 <Process(Process-4, stopped[SIGTERM])>]

In [12]:
#jobs[3].terminate()

In [13]:
jobs = []

In [15]:
from lib.charting_lib import *

def freedom_chart(symbol):
    ohlc_df = pd.read_json(redis_conn.get(symbol), orient='columns')
    ohlc_df.index.rename('date', inplace=True)
    
    trade_df = pd.read_json(redis_conn.get(symbol+cache_postfix+'Trade'), orient='columns')
    return render_charts(ohlc_df, trade_df, symbol)

fig = freedom_chart('TCS')
fig

In [ ]:
temp_file = pd.HDFStore("data/kite_cache.h5", mode="r")
temp_file.keys()

In [ ]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


## Redis Console

In [ ]:
from redis import Redis
import json

conn = Redis(host='redis', port=6379, db=0, charset="utf-8", decode_responses=True)

In [ ]:
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
# Unit Test#1
freq='day'
exchange='NSE'
fromDate = '2019-01-01'
toDate = '2019-05-03'
stock='INFY'

state = 'INIT'
hash_key = stock+'_state'

try:
    all_keys = list(conn.hgetall(hash_key).keys())
    conn.hdel(hash_key,*all_keys)
except:
    pass

#conn.hmset(hash_key, {'state':state,'stock':stock, 'qty':0,'price':0,'algo':'','freq':'day','so':0,'target':0})

msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)


In [ ]:
# Unit Test#2
stock = 'NDTV'
msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)
conn.xadd('msgBufferQueue',{'msg': 'next'})

In [ ]:
# Shut down
conn.publish('kite_simulator','stop')
conn.publish('trade_handler','stop')
freedom.job.terminate()
print(freedom.job.is_alive())
conn.pubsub_channels()

In [ ]:
conn.pubsub_channels()

In [ ]:
# Debug
stock='TCS'
#notification_despatcher(None, 'msg')
print(stock)
pd.read_json(conn.hget(stock+'_state','ohlc'))

In [ ]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

## Scratchpad

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objs as go
from datetime import datetime as dt
from datetime import timedelta
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from lib.multitasking_lib import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [7]:
# GetData

logger.setLevel(logging.DEBUG)
stock = 'WIPRO'
fromDate = '2019-2-2'
toDate  = '2019-2-28'
freq = 'day'
mydate = fromDate 
exchange = 'NSE'
ohlc_data = {}
ohlc_data[stock] = getData(stock, fromDate, toDate, exchange, freq, False, stock)

In [8]:
row = ohlc_data[stock].tail(1)

In [9]:
row

,close,high,low,open,volume
date,,,,,
2019-02-28,277.37,283.23,275.64,281.88,9428265


In [ ]:
import numpy as np
no = 1
for i in np.linspace(0,no-1,no):
    i = int(i)
    row = ohlc_data[stock].iloc[i]
    print(type(row))
    index = ohlc_data[stock].index[i]
    update_plot_cache(stock, row)
    print(index)
    print(ohlc_data[stock].iloc[i])
    # Construct Json message like Kite
    mydate = "{}-{}-{}".format(index.year,index.month,index.day)        
    msg = {exchange+":"+stock:{"ohlc":{'date':mydate,'open':row['open'],'high':row['high'],'low':row['low'],'close':row['close'],'volume':row['volume']}}}
    pdebug(msg)
    msg = json.dumps(msg)

In [ ]:
for i,r in ohlc_data[stock].iterrows():
    print(type(r))
    print(index)
    print(r)
    break

In [ ]:
#tmp_df = pd.DataFrame({'buy': ohlc_data.iloc[1:5]['close'], 'sell':ohlc_data.iloc[10:15]['close']})

tmp_df_s = pd.DataFrame()
tmp_df_b = pd.DataFrame()
tmp_df_b['buy'] = ohlc_data.iloc[-1:]['close']
tmp_df_s['sell'] = ohlc_data.iloc[20:25]['close']

tmp_df_b.append(tmp_df_s)

In [ ]:
msg = {exchange+":"+stock:{"ohlc":{'date':mydate,'open':row['open'],'high':row['high'],'low':row['low'],'close':row['close'],'volume':row['volume']}}}            

In [3]:
import pandas as pd
import numpy as np

In [19]:
tmp = pd.DataFrame(data=np.linspace(1,10,10), columns=["val"])

In [13]:
tmp["val"].max()

10.0

In [15]:
if "val2" in tmp:
    print("yes")

In [20]:
[tmp["val"].max() for i in np.linspace(1,tmp.shape[0],tmp.shape[0])]

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

In [23]:
tmp.index.shape[0]

10